## Notebook N. 1

Exploratory Data Analysis and initial Feature Engineering

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("../data/Medical Appointments.csv")

### Description

PatientId: unique patient ID

AppointmentID: unique appointment ID

Gender: patient’s gender

ScheduledDay: when the appointment was scheduled

AppointmentDay: date of the appointment

Age: patient’s age

Neighbourhood: patient’s neighbourhood (in Brazil)

Scholarship: whether the patient benefited from social welfare to pay for the appointment

Hipertension: whether the patient has hypertension

Diabetes: whether the patient has diabetes

Alcoholism: whether the patient is an alcoholic

Handcap: whether the patient is handicapped

SMS_received: whether the patient received a reminder by text message

No-show: whether they actually showed up to the appointment

In [ ]:
df.head()

There are no null cases

In [ ]:
df.info()

In [ ]:
import pandas_profiling as pp
pp.ProfileReport(df)

Using pandas_profiling we can conclude that:
    - There are no missing values
    - Age has a minimum value of -1 and a maximum of 115. The most frequent value is 0.
    - Handcap has 5 different values, but must of them are 0 or 1.
    - Neighbourhood has 81 different values.
    - The positive class is 20,2%.
    

## Change dtype of variables

In [ ]:
df["AppointmentDay"] = pd.to_datetime(df["AppointmentDay"])

In [ ]:
df["Gender"] = df["Gender"].replace({"F":0,"M":1})

In [ ]:
df["No-show"] = df["No-show"].replace({"No":0,"Yes":1})

In [ ]:
df["ScheduledDay"] = pd.to_datetime(df["ScheduledDay"])

## Generate new columns

In [ ]:
df.head()

#### Generate time related variables

Time difference between appointment and schedule

In [ ]:
df["DaysDiff"] = df["AppointmentDay"] - df["ScheduledDay"]

Difference expressed in minutes

In [ ]:
df["MinutesDiff"] = pd.to_numeric(df["DaysDiff"].dt.total_seconds()/60)

Flag that indicates whether appointment and scheduled were on the same day

In [ ]:
df["ScheduledSameDay"] = (df["AppointmentDay"].dt.date == df["ScheduledDay"].dt.date).replace({True:1,False:0})

Day of week of Appointment

In [ ]:
# It is assumed the week starts on Monday, which is denoted by 0 and ends on Sunday which is denoted by 6.
df["AppointmentDayofWeek"] = df["AppointmentDay"].dt.dayofweek

Day of week of Scheduled


In [ ]:
df["ScheduledDayofWeek"] = df["ScheduledDay"].dt.dayofweek

Hour of scheduled

In [ ]:
df["ScheduledHour"] = df["ScheduledDay"].dt.hour

Month of appointment

In [ ]:
df["AppointmentMonth"] = df["AppointmentDay"].dt.month

#### Generate other variables

In [ ]:
df = df.sort_values("ScheduledDay")

In [ ]:
patient_groups = df.groupby("PatientId")

Count number of times a patient had had an appointment before and if showed or not

In [ ]:
total_appointments = []
ids = []
patient = []
total_not_showed = []

for i,j in patient_groups:
    appointments = 0
    not_showed = 0
    for h in range(j.shape[0]):
        total_appointments.append(appointments)
        total_not_showed.append(not_showed)
        ids.append(j.iloc[h,:].name)
        if j.iloc[h,:]["No-show"] == 1:
            not_showed += 1
        appointments += 1

In [ ]:
appointments_counter = pd.DataFrame(ids, columns = ["RowIndex"])
appointments_counter["AppointmentsCount"] = total_appointments
appointments_counter["NotShowedCount"] = total_not_showed
appointments_counter["ProportionNotShowed"] = appointments_counter["NotShowedCount"]/appointments_counter["AppointmentsCount"]
appointments_counter["ProportionNotShowed"] = appointments_counter["ProportionNotShowed"].fillna(-9999)

In [ ]:
appointments_counter.sample(10)

In [ ]:
df = df.reset_index()

In [ ]:
df = pd.merge(df, appointments_counter, left_on="index", right_on="RowIndex")

Flag that indicates whether the patient had had an appointment before

In [ ]:
df["FirstAppointment"] = (df["AppointmentsCount"] == 0).replace({True:1, False:0})

## Data visualization

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
init_notebook_mode(connected=True)

Apparently, there is a little bit more chance that you don't show if you scheduled at morning

In [ ]:
not_showed_hour = df["ScheduledHour"][df["No-show"]==1]
showed_hour = df["ScheduledHour"][df["No-show"]==0]
traceA = go.Histogram(x = not_showed_hour, histnorm="probability", name = "no-show")
traceB = go.Histogram(x = showed_hour, histnorm="probability", name = "show")
data = [traceA,traceB]
layout = go.Layout(title='Probability of showing by hour of schedule')
fig = go.Figure(data=data, layout=layout)
iplot(fig)


The day of week is not relevant

In [ ]:
not_showed_hour = df["AppointmentDayofWeek"][df["No-show"]==1]
showed_hour = df["AppointmentDayofWeek"][df["No-show"]==0]
traceA = go.Histogram(x = not_showed_hour, histnorm="probability")
traceB = go.Histogram(x = showed_hour, histnorm="probability")

iplot([traceA,traceB])

Most people that doesn't show scheduled a different day

In [ ]:
not_showed_hour = df["ScheduledSameDay"][df["No-show"]==1]
showed_hour = df["ScheduledSameDay"][df["No-show"]==0]
traceA = go.Histogram(x = not_showed_hour, histnorm="probability", name = "no-show")
traceB = go.Histogram(x = showed_hour, histnorm="probability", name = "show")

layout = go.Layout(
    xaxis=go.XAxis(
        ticktext=["Scheduled a different day","Scheduled same day"],
        tickvals=[0, 1]
    ))

figure = go.Figure(data = [traceA, traceB], layout = layout)

iplot(figure,layout)

In [ ]:
not_showed_hour = df["NotShowedCount"][df["No-show"]==1]
showed_hour = df["NotShowedCount"][df["No-show"]==0]
traceA = go.Histogram(x = not_showed_hour, histnorm="probability")
traceB = go.Histogram(x = showed_hour, histnorm="probability")

iplot([traceA,traceB])

In [ ]:

#dataA = df["Age"][df["Gender"] == 1]
#dataB = df["Age"][df["Gender"] == 0]
#traceA = go.Violin(x = df["No-show"][df["Gender"] == 1],y = dataA, side = "negative", name = "M")
#traceB = go.Violin(x = df["No-show"][df["Gender"] == 0],y = dataB, side = "positive", name = "F")

traceA = go.Violin(x = df["No-show"],y = df["Age"],)

#traceB = go.Violin(dataB)
iplot([traceA])

In [ ]:
dataA = df["Age"][df["No-show"] == 1]
dataB = df["Age"][df["No-show"] == 0]
x_data_A = df["Gender"][df["No-show"] == 1].copy()
x_data_B = df["Gender"][df["No-show"] == 0].copy()

traceA = go.Violin(x = x_data_A,y = dataA, side = "negative", name = "No-show")
traceB = go.Violin(x = x_data_B,y = dataB, side = "positive", name = "Show")

layout = go.Layout(
    xaxis=go.XAxis(
        ticktext=["F","M"],
        tickvals=[0, 1]
    ))


figure = go.Figure(data = [traceA, traceB], layout = layout)
iplot(figure)

In [ ]:
corr_matrix = np.corrcoef(df.drop(["index","PatientId","AppointmentID", "ScheduledDay","AppointmentDay","Neighbourhood","DaysDiff","RowIndex", 'ProportionNotShowed', 'FirstAppointment'], axis=1),rowvar=False)
columns = df.drop(["index","PatientId","AppointmentID", "ScheduledDay","AppointmentDay","Neighbourhood","DaysDiff","RowIndex",'ProportionNotShowed', 'FirstAppointment'], axis=1).columns

In [ ]:
columns

In [ ]:
traceA = go.Heatmap(z=corr_matrix, x=columns, y = columns)
layout = go.Layout(yaxis=go.YAxis(automargin=True), xaxis=go.XAxis(automargin=True))
figure = go.Figure(data = [traceA],layout = layout)
iplot(figure)

In [ ]:
gender_ctab = pd.crosstab(df["Gender"], df["No-show"], normalize=True)
gender_odds = gender_ctab[1]/gender_ctab[0] # Odds
gender_odds

In [ ]:
sms_ctab = pd.crosstab(df["SMS_received"], df["No-show"], normalize=True)
sms_odds = sms_ctab[1]/sms_ctab[0] # Odds
sms_odds

In [ ]:
same_ctab = pd.crosstab(df["ScheduledSameDay"], df["No-show"], normalize=True)
same_odds = same_ctab[1]/same_ctab[0] # Odds
same_odds

In [ ]:

df["No-show"].value_counts(normalize=True)
traceA = go.Pie(labels=["show","no-show"], values=[0.8,0.2])
iplot([traceA])

#### Neighbourhood
Some neighbourhoods are much more important than others

In [ ]:
neighbourhood_ctab = pd.crosstab(df["Neighbourhood"], df["No-show"], normalize=True)
neighbourhood_odds = neighbourhood_ctab[1]/neighbourhood_ctab[0] # Odds

In [ ]:
neighbourhood_odds.sort_values(ascending=False).head(10)

In [ ]:
neighbourhood_odds.sort_values(ascending=False).tail(10)

### Further transformations
Finally, we will finish preparing our dataset to try different models

In [ ]:
df["Handcap"].value_counts()

In [ ]:
df["Handcap"][df["Handcap"] > 1] = 1

In [ ]:
df.drop(["index","PatientId","AppointmentID","ScheduledDay","RowIndex","DaysDiff"], inplace=True, axis=1)


### Conclusion
Most variables are not really correlated with the target so we will need to try non linear transformations. The most linearly correlated variables are:
- SMS_received
- MinutesDiff
- ScheduledSameDay

In addition, some variables are not really linearly correlated but might have some information:
- ScheduledHour
- NotShowedCount
- Age

Apart from that, some neighbourhoods seems to be important for predicting purposes.

In [ ]:
df.to_csv("cleaned_data.csv",index=False)